In [ ]:
from google.colab import drive
drive.mount('/content/drive2', force_remount=True)
driveHome = "/content/drive2/MyDrive/MasterThesis"

Mounted at /content/drive2


In [ ]:
# @title run this cell and RESTART RUNTIME (may last quite long)
%%capture

!pip install pillow==9.0.0
!pip install tensorflow==2.8.0
!pip install --upgrade numpy scipy h5py matplotlib opencv-python

!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
def flushDrive():
    drive.flush_and_unmount()
    drive.mount('/content/drive2')

In [ ]:
#@title Imports

import tensorflow as tf
import tensorflow.keras.applications

#from imageai.Prediction.Custom import ModelTraining

import os
from os.path import join, basename
from pathlib import Path
#import moviepy.editor
import cv2
import shutil
import re
from collections import defaultdict
from PIL import Image
from distutils import dir_util
from datetime import datetime
import pytz

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
UcfPath = join(driveHome, "UCF-SmartSplit")
framesGroups = os.path.join(UcfPath, "frames")

In [ ]:
classes = os.scandir(os.path.join(framesGroups, os.listdir(framesGroups)[0], "train"))
classes = [dir.name for dir in classes]
print(classes)
print(len(classes))

['BlowingCandles', 'Billiards', 'BrushingTeeth', 'CuttingInKitchen', 'Mixing', 'MoppingFloor', 'ShavingBeard', 'Swing', 'Typing', 'WallPushups']
10


In [ ]:
#@title ImageAI

import tensorflow as tf
from PIL import Image
import time
import numpy as np
import os
import warnings
from matplotlib.cbook import deprecated
import json

class ClassificationModelTrainer:
    """
        This is the Classification Model training class, that allows you to define a deep learning network
        from the 4 available networks types supported by ImageAI which are MobileNetv2, ResNet50,
        InceptionV3 and DenseNet121.
    """

    def __init__(self):
        self.__modelType = ""
        self.__use_pretrained_model = False
        self.__data_dir = ""
        self.__train_dir = ""
        self.__test_dir = ""
        self.__logs_dir = ""
        self.__num_epochs = 10
        self.__trained_model_dir = ""
        self.__model_class_dir = ""
        self.__initial_learning_rate = 1e-3
        self.__model_collection = []


    def setModelTypeAsSqueezeNet(self):
        raise ValueError("ImageAI no longer support SqueezeNet. You can use MobileNetV2 instead by downloading the MobileNetV2 model and call the function 'setModelTypeAsMobileNetV2'")

    def setModelTypeAsMobileNetV2(self):
        """
        'setModelTypeAsMobileNetV2()' is used to set the model type to the MobileNetV2 model
        for the training instance object .
        :return:
        """
        self.__modelType = "mobilenetv2"

    @deprecated(since="2.1.6", message="'.setModelTypeAsResNet()' has been deprecated! Please use 'setModelTypeAsResNet50()' instead.")
    def setModelTypeAsResNet(self):
        return self.setModelTypeAsResNet50()

    def setModelTypeAsResNet50(self):
        """
         'setModelTypeAsResNet()' is used to set the model type to the ResNet model
                for the training instance object .
        :return:
        """
        self.__modelType = "resnet50"

    
    @deprecated(since="2.1.6", message="'.setModelTypeAsDenseNet()' has been deprecated! Please use 'setModelTypeAsDenseNet121()' instead.")
    def setModelTypeAsDenseNet(self):
        return self.setModelTypeAsDenseNet121()

    def setModelTypeAsDenseNet121(self):
        """
         'setModelTypeAsDenseNet()' is used to set the model type to the DenseNet model
                for the training instance object .
        :return:
        """
        self.__modelType = "densenet121"

    def setModelTypeAsEfficientNetV2B0(self):
        """
         'setModelTypeAsDenseNet()' is used to set the model type to the DenseNet model
                for the training instance object .
        :return:
        """
        self.__modelType = "efficientnetv2b0"

    def setModelTypeAsInceptionV3(self):
        """
         'setModelTypeAsInceptionV3()' is used to set the model type to the InceptionV3 model
                for the training instance object .
        :return:
        """
        self.__modelType = "inceptionv3"

    def setDataDirectory(self, data_directory="", train_subdirectory="train", test_subdirectory="test",
                         models_subdirectory="models", json_subdirectory="json"):
        """
        'setDataDirectory()'

        - data_directory , is required to set the path to which the data/dataset to be used for
                 training is kept. The directory can have any name, but it must have 'train' and 'test'
                 sub-directory. In the 'train' and 'test' sub-directories, there must be sub-directories
                 with each having it's name corresponds to the name/label of the object whose images are
                to be kept. The structure of the 'test' and 'train' folder must be as follows:

                >> train >> class1 >> class1_train_images
                         >> class2 >> class2_train_images
                         >> class3 >> class3_train_images
                         >> class4 >> class4_train_images
                         >> class5 >> class5_train_images

                >> test >> class1 >> class1_test_images
                        >> class2 >> class2_test_images
                        >> class3 >> class3_test_images
                        >> class4 >> class4_test_images
                        >> class5 >> class5_test_images

        - train_subdirectory (optional), subdirectory within 'data_directory' where the training set is. Defaults to 'train'.
        - test_subdirectory (optional), subdirectory within 'data_directory' where the testing set is. Defaults to 'test'.
        - models_subdirectory (optional), subdirectory within 'data_directory' where the output models will be saved. Defaults to 'models'.
        - json_subdirectory (optional), subdirectory within 'data_directory' where the model classes json file will be saved. Defaults to 'json'.

        :param data_directory:
        :param train_subdirectory:
        :param test_subdirectory:
        :param models_subdirectory:
        :param json_subdirectory:
        :return:
        """

        self.__data_dir = data_directory

        self.__train_dir = os.path.join(self.__data_dir, train_subdirectory)
        self.__test_dir = os.path.join(self.__data_dir, test_subdirectory)
        self.__trained_model_dir = os.path.join(self.__data_dir, models_subdirectory)
        self.__model_class_dir = os.path.join(self.__data_dir, json_subdirectory)
        self.__logs_dir = os.path.join(self.__data_dir, "logs")

    def lr_schedule(self, epoch):

        # Learning Rate Schedule


        lr = self.__initial_learning_rate
        total_epochs = self.__num_epochs

        check_1 = int(total_epochs * 0.9)
        check_2 = int(total_epochs * 0.8)
        check_3 = int(total_epochs * 0.6)
        check_4 = int(total_epochs * 0.4)

        if epoch > check_1:
            lr *= 1e-4
        elif epoch > check_2:
            lr *= 1e-3
        elif epoch > check_3:
            lr *= 1e-2
        elif epoch > check_4:
            lr *= 1e-1


        return lr




    def trainModel(self, num_objects, num_experiments=200, enhance_data=False, batch_size = 32, initial_learning_rate=1e-3, show_network_summary=False, training_image_size = 224, 
                   continue_from_model=None, transfer_from_model=None, transfer_with_full_training=True, initial_num_objects = None, save_full_model = True):

        """
        'trainModel()' function starts the model actual training. It accepts the following values:
        - num_objects , which is the number of classes present in the dataset that is to be used for training
        - num_experiments , also known as epochs, it is the number of times the network will train on all the training dataset
        - enhance_data (optional) , this is used to modify the dataset and create more instance of the training set to enhance the training result
        - batch_size (optional) , due to memory constraints, the network trains on a batch at once, until all the training set is exhausted. The value is set to 32 by default, but can be increased or decreased depending on the meormory of the compute used for training. The batch_size is conventionally set to 16, 32, 64, 128.
        - initial_learning_rate(optional) , this value is used to adjust the weights generated in the network. You rae advised to keep this value as it is if you don't have deep understanding of this concept.
        - show_network_summary(optional) , this value is used to show the structure of the network should you desire to see it. It is set to False by default
        - training_image_size(optional) , this value is used to define the image size on which the model will be trained. The value is 224 by default and is kept at a minimum of 100.
        - continue_from_model (optional) , this is used to set the path to a model file trained on the same dataset. It is primarily for continuos training from a previously saved model.
        - transfer_from_model (optional) , this is used to set the path to a model file trained on another dataset. It is primarily used to perform tramsfer learning.
        - transfer_with_full_training (optional) , this is used to set the pre-trained model to be re-trained across all the layers or only at the top layers.
        - initial_num_objects (required if 'transfer_from_model' is set ), this is used to set the number of objects the model used for transfer learning is trained on. If 'transfer_from_model' is set, this must be set as well.
        - save_full_model ( optional ), this is used to save the trained models with their network types. Any model saved by this specification can be loaded without specifying the network type.


        :param num_objects:
        :param num_experiments:
        :param enhance_data:
        :param batch_size:
        :param initial_learning_rate:
        :param show_network_summary:
        :param training_image_size:
        :param continue_from_model:
        :param transfer_from_model:
        :param initial_num_objects:
        :param save_full_model:
        :return:
        """
        self.__num_epochs = num_experiments
        self.__initial_learning_rate = initial_learning_rate
        lr_scheduler = tf.keras.callbacks.LearningRateScheduler(self.lr_schedule)


        if(training_image_size < 100):
            warnings.warn("The specified training_image_size {} is less than 100. Hence the training_image_size will default to 100.".format(training_image_size))
            training_image_size = 100



        if (self.__modelType == "mobilenetv2"):
            if (continue_from_model != None):
                model = tf.keras.applications.MobileNetV2(input_shape=(training_image_size, training_image_size, 3), weights=continue_from_model, classes=num_objects,
                include_top=True)
                if (show_network_summary == True):
                    print("Training using weights from a previouly model")
            elif (transfer_from_model != None):
                base_model = tf.keras.applications.MobileNetV2(input_shape=(training_image_size, training_image_size, 3), weights= transfer_from_model,
                include_top=False, pooling="avg")

                base_model.trainable = False

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

                if (show_network_summary == True):
                    print("Training using weights from a pre-trained ImageNet model")
            else:
                base_model = tf.keras.applications.MobileNetV2(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")
                
                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

        elif (self.__modelType == "resnet50"):
            if (continue_from_model != None):
                model = tf.keras.applications.ResNet50(input_shape=(training_image_size, training_image_size, 3), weights=continue_from_model, classes=num_objects,
                include_top=True)
                if (show_network_summary == True):
                    print("Training using weights from a previouly model")
            elif (transfer_from_model != None):
                base_model = tf.keras.applications.ResNet50(input_shape=(training_image_size, training_image_size, 3), weights= transfer_from_model,
                include_top=False, pooling="avg")

                base_model.trainable = False

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

                if (show_network_summary == True):
                    print("Training using weights from a pre-trained ImageNet model")
            else:
                base_model = tf.keras.applications.ResNet50(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

        elif (self.__modelType == "inceptionv3"):
            if (continue_from_model != None):
                model = tf.keras.applications.InceptionV3(input_shape=(training_image_size, training_image_size, 3), weights=continue_from_model, classes=num_objects,
                include_top=True)
                if (show_network_summary == True):
                    print("Training using weights from a previouly model")
            elif (transfer_from_model != None):
                base_model = tf.keras.applications.InceptionV3(input_shape=(training_image_size, training_image_size, 3), weights= transfer_from_model,
                include_top=False, pooling="avg")

                base_model.trainable = False

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

                if (show_network_summary == True):
                    print("Training using weights from a pre-trained ImageNet model")
            else:
                base_model = tf.keras.applications.InceptionV3(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

            base_model = tf.keras.applications.InceptionV3(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")
            
        elif (self.__modelType == "efficientnetv2b0"):
            if (continue_from_model != None):
                model = tf.keras.applications.EfficientNetV2B0(input_shape=(training_image_size, training_image_size, 3), weights=continue_from_model, classes=num_objects,
                include_top=True)
                if (show_network_summary == True):
                    print("Training using weights from a previouly model")
            elif (transfer_from_model != None):
                base_model = tf.keras.applications.EfficientNetV2B0(input_shape=(training_image_size, training_image_size, 3), weights= transfer_from_model,
                include_top=False, pooling="avg")

                base_model.trainable = False

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

                if (show_network_summary == True):
                    print("Training using weights from a pre-trained ImageNet model")
            else:
                base_model = tf.keras.applications.EfficientNetV2B0(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

            base_model = tf.keras.applications.EfficientNetV2B0(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")

        elif (self.__modelType == "densenet121"):
            if (continue_from_model != None):
                model = tf.keras.applications.DenseNet121(input_shape=(training_image_size, training_image_size, 3), weights=continue_from_model, classes=num_objects,
                include_top=True)
                if (show_network_summary == True):
                    print("Training using weights from a previouly model")
            elif (transfer_from_model != None):
                base_model = tf.keras.applications.DenseNet121(input_shape=(training_image_size, training_image_size, 3), weights= transfer_from_model,
                include_top=False, pooling="avg")

                base_model.trainable = False

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

                if (show_network_summary == True):
                    print("Training using weights from a pre-trained ImageNet model")
            else:
                base_model = tf.keras.applications.DenseNet121(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")

                network = base_model.output
                network = tf.keras.layers.Dense(num_objects, activation='softmax',
                         use_bias=True)(network)
                
                model = tf.keras.Model(inputs=base_model.input, outputs=network)

            base_model = tf.keras.applications.DenseNet121(input_shape=(training_image_size, training_image_size, 3), weights= None, classes=num_objects,
                include_top=False, pooling="avg")


        optimizer = tf.keras.optimizers.Adam(learning_rate=self.__initial_learning_rate, decay=1e-4)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        if (show_network_summary == True):
            model.summary()

        model_name = 'model_ex-{epoch:03d}_loss-{val_loss:03f}_acc-{val_accuracy:03f}.h5'

        log_name = '{}_lr-{}_{}'.format(self.__modelType, initial_learning_rate, time.strftime("%Y-%m-%d-%H-%M-%S"))

        if not os.path.isdir(self.__trained_model_dir):
            os.makedirs(self.__trained_model_dir)

        if not os.path.isdir(self.__model_class_dir):
            os.makedirs(self.__model_class_dir)

        if not os.path.isdir(self.__logs_dir):
            os.makedirs(self.__logs_dir)

        model_path = os.path.join(self.__trained_model_dir, model_name)


        logs_path = os.path.join(self.__logs_dir, log_name)
        if not os.path.isdir(logs_path):
            os.makedirs(logs_path)

        save_weights_condition = True

        if(save_full_model == True ):
            save_weights_condition = False


        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_path,
                                     monitor='val_accuracy',
                                     verbose=1,
                                     save_weights_only=save_weights_condition,
                                     save_best_only=True)


        tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_path, 
                                  histogram_freq=0, 
                                  write_graph=False, 
                                  write_images=False)
        
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                min_delta=0.005,
                                patience=10,
                                verbose=1)

        

        if (enhance_data == True):
            print("Using Enhanced Data Generation")

        height_shift = 0
        width_shift = 0
        if (enhance_data == True):
            height_shift = 0.1
            width_shift = 0.1

        if (self.__modelType == "efficientnetv2b0"):
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
            test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
        else:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
            test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

        train_generator = train_datagen.flow_from_directory(self.__train_dir, target_size=(training_image_size, training_image_size),
                                                            batch_size=batch_size,
                                                            class_mode="categorical")
        test_generator = test_datagen.flow_from_directory(self.__test_dir, target_size=(training_image_size, training_image_size),
                                                          batch_size=batch_size,
                                                          class_mode="categorical")

        class_indices = train_generator.class_indices
        class_json = {}
        for eachClass in class_indices:
            class_json[str(class_indices[eachClass])] = eachClass

        with open(os.path.join(self.__model_class_dir, "model_class.json"), "w+") as json_file:
            json.dump(class_json, json_file, indent=4, separators=(",", " : "),
                      ensure_ascii=True)
            json_file.close()
        print("JSON Mapping for the model classes saved to ", os.path.join(self.__model_class_dir, "model_class.json"))

        num_train = len(train_generator.filenames)
        num_test = len(test_generator.filenames)
        print("Number of experiments (Epochs) : ", self.__num_epochs)

        
        model.fit(x=train_generator, steps_per_epoch=int(num_train / batch_size), epochs=self.__num_epochs,
                            validation_data=test_generator,
                            validation_steps=int(num_test / batch_size), callbacks=[checkpoint, lr_scheduler, callback])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: MatplotlibDeprecationWarning: 
The deprecated function was deprecated in Matplotlib 3.4 and will be removed two minor releases later.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: MatplotlibDeprecationWarning: 
The deprecated function was deprecated in Matplotlib 3.4 and will be removed two minor releases later.


In [ ]:
localData = "/home/local/"

In [ ]:
models = ["InceptionV3Model", "MobileNetV2Model", "EfficientNetV2B0Model"]

In [ ]:
def get_best_worse_models(path):
    acc_and_path = {}
    for file_name in os.listdir(path):
        file_path = join(path, file_name)
        found = re.search(r"acc-0\.\d+.h5", file_name).group()
        acc = found[4 : len(found)-3]
        acc_and_path[float(acc)] = file_path
    
    acc_and_path = [path for key, path in sorted(acc_and_path.items(), reverse=True)]

    if len(acc_and_path) > 1:
        return acc_and_path[0], acc_and_path[1:]
    else:
        return acc_and_path[0], []

In [ ]:
def fitModel(modelName, dataFolder, saveTo, transfer):
    if transfer:
        saveTo=join(saveTo, "-with-transfer", modelName, basename(dataFolder))
    else:
        saveTo=join(saveTo, "-no-transfer", modelName, basename(dataFolder))

    batch_size = 128
        
    max_epochs = 50
    known_classes = len(classes)

    if os.path.exists(saveTo):
        print(f"Path: {saveTo} exists - skipping")
        return

    imagesPerFolder = basename(dataFolder)
    localFrames = join(localData, modelName, imagesPerFolder)
    validationConstPath = join(localData, "validation-constant")
    if not os.path.exists(localFrames):
        shutil.copytree(join(dataFolder, "train"), join(localFrames, "train"))
    if not os.path.exists(validationConstPath):
        shutil.copytree(join(UcfPath, "validation-constant"), validationConstPath)

    if os.path.exists(join(localFrames, "train")) and os.path.exists(validationConstPath):
        if len(os.listdir(join(localFrames, "train"))) + len(os.listdir(validationConstPath)) < 2 * known_classes:
                print("Not enough class folders in either train or test")
                return                                                           

        for action in os.scandir(join(localFrames, "train")):
            if len(os.listdir(action.path)) < int(imagesPerFolder):
                print(f"Not sufficient train folder content ({basename(action.path)})")
                return

        for action in os.scandir(validationConstPath):
            if len(os.listdir(action.path)) < int(imagesPerFolder) * 0.2:
                print(f"Not sufficient test folder content ({basename(action.path)})")
                return
    
    print(imagesPerFolder)
    if transfer:
        print("\n-With Transfer-")
    else:
        print("\n-No Transfer-")

    trainer = ClassificationModelTrainer()
    if (modelName == "DenseNet121Model"): trainer.setModelTypeAsDenseNet121()
    if (modelName == "InceptionV3Model"): trainer.setModelTypeAsInceptionV3()
    if (modelName == "MobileNetV2Model"): trainer.setModelTypeAsMobileNetV2()
    if (modelName == "ResNet50Model"): trainer.setModelTypeAsResNet50()
    if (modelName == "EfficientNetV2B0Model"): trainer.setModelTypeAsEfficientNetV2B0()
    trainer.setDataDirectory(data_directory="", train_subdirectory=join(localFrames, "train"), test_subdirectory=validationConstPath, 
                             models_subdirectory=join(localFrames, "models"))

    if transfer:
        trainer.trainModel(num_objects=known_classes, transfer_from_model="imagenet",  batch_size=batch_size, num_experiments=max_epochs, enhance_data=False,  show_network_summary=False)
    else:
        trainer.trainModel(num_objects=known_classes, batch_size=batch_size, num_experiments=max_epochs, enhance_data=False,  show_network_summary=False)

    Path(saveTo).mkdir(parents=True, exist_ok=True)
    best, _ = get_best_worse_models(join(localFrames, "models"))
    shutil.move(best, join(saveTo, basename(best)))
    shutil.rmtree(join(localFrames, "models"))

    flushDrive()

In [ ]:
for modelName in models:
    print("\n---####---")
    print(modelName)
    print("---####---")
    for pictureCount in sorted(os.scandir(framesGroups), key = lambda n: int(n.name), reverse=True):
        fitModel(modelName, pictureCount.path, join(driveHome, "taught-smart"), False)
        fitModel(modelName, pictureCount.path, join(driveHome, "taught-smart"), True)


---####---
InceptionV3Model
---####---
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/InceptionV3Model/3000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/InceptionV3Model/3000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/InceptionV3Model/2000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/InceptionV3Model/2000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/InceptionV3Model/1500 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/InceptionV3Model/1500 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/InceptionV3Model/1000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/InceptionV3Model/1000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/InceptionV3

In [ ]:
UcfAugmented = os.path.join(UcfPath, "augmented")
UcfAugmentExtract = os.path.join(UcfPath, "augmentation-prep")

In [ ]:
def fitModelWithAugmentation(modelName, dataFolder, saveTo, transfer):
    split = dataFolder.rsplit("/", 3)
    perc, augmentationName, quan = split[len(split)-3], split[len(split)-2], split[len(split)-1]
    if transfer:
        saveTo=join(saveTo, "-with-transfer", "Augmented", perc, augmentationName, modelName, quan)
    else:
        saveTo=join(saveTo, "-no-transfer", "Augmented", perc, augmentationName, modelName, quan)

    batch_size = 128

    max_epochs = 50
    known_classes = len(classes)

    if os.path.exists(saveTo):
        print(f"Path: {saveTo} exists - skipping")
        return

    def verbose():
        print(f"\n\n---{perc}---{augmentationName}---{modelName}---{quan}---")
        if transfer:
            print("\n-With Transfer-")
        else:
            print("-No Transfer-")

    localFrames = join(localData, perc, augmentationName, modelName, quan)
    validationConstPath = "/home/data/validation-constant"
    if not os.path.exists(localFrames):
        verbose()
        shutil.copytree(dataFolder, join(localFrames, "train"))
        dir_util.copy_tree(join(UcfAugmentExtract, perc, quan), join(localFrames, "train"))
    else:
        verbose()

    if not os.path.exists(validationConstPath):
            shutil.copytree(join(UcfPath, "validation-constant"), validationConstPath)

    if os.path.exists(join(localFrames, "train")) and os.path.exists(validationConstPath):
        if len(os.listdir(join(localFrames, "train"))) + len(os.listdir(validationConstPath)) < 2 * known_classes:
                print("Not enough class folders in either train or test")
                return                                                           

        for action in os.scandir(join(localFrames, "train")):
            if len(os.listdir(action.path)) < int(quan) - 1:
                print(f"Not sufficient train folder content ({basename(action.path)}) vs. {len(os.listdir(action.path))} < {int(quan)}")
                return

        for action in os.scandir(validationConstPath):
            if len(os.listdir(action.path)) < int(quan) * 0.2:
                print(f"Not sufficient test folder content ({basename(action.path)}) vs. {len(os.listdir(action.path))} < {int(quan)}")
                return

    trainer = ClassificationModelTrainer()
    if (modelName == "DenseNet121Model"): trainer.setModelTypeAsDenseNet121()
    if (modelName == "InceptionV3Model"): trainer.setModelTypeAsInceptionV3()
    if (modelName == "MobileNetV2Model"): trainer.setModelTypeAsMobileNetV2()
    if (modelName == "ResNet50Model"): trainer.setModelTypeAsResNet50()
    if (modelName == "EfficientNetV2B0Model"): trainer.setModelTypeAsEfficientNetV2B0()
    
    trainer.setDataDirectory(data_directory="", train_subdirectory=join(localFrames, "train"), test_subdirectory=validationConstPath, 
                                models_subdirectory=join(localFrames, "models"))

    if transfer:
        trainer.trainModel(num_objects=known_classes, transfer_from_model="imagenet",  batch_size=batch_size, num_experiments=max_epochs, enhance_data=False,  show_network_summary=False)
    else:
        trainer.trainModel(num_objects=known_classes, batch_size=batch_size, num_experiments=max_epochs, enhance_data=False,  show_network_summary=False)

    Path(saveTo).mkdir(parents=True, exist_ok=True)
    best, _ = get_best_worse_models(join(localFrames, "models"))
    shutil.move(best, join(saveTo, basename(best)))
    shutil.rmtree(join(localFrames, "models"))

    print(datetime.now().strftime("%H:%M:%S"))
    flushDrive()
    print("Flushed")

In [ ]:
for modelName in models:
    for perc in os.listdir(UcfAugmented):
        percDir = join(UcfAugmented, perc)
        for operation in os.listdir(percDir):
            operationDir = join(percDir, operation)
            for quan in os.listdir(operationDir):
                source = join(operationDir, quan)
                fitModelWithAugmentation(modelName, source, join(driveHome, "taught-smart"), False)
                fitModelWithAugmentation(modelName, source, join(driveHome, "taught-smart"), True)

Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/Augmented/0.333/mix-net/InceptionV3Model/500 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.333/mix-net/InceptionV3Model/500 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/Augmented/0.333/mix-net/InceptionV3Model/2000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.333/mix-net/InceptionV3Model/2000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/Augmented/0.333/mix-net/InceptionV3Model/1000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.333/mix-net/InceptionV3Model/1000 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-no-transfer/Augmented/0.333/mix-net/InceptionV3Model/1500 exists - skipping
Path: /content/drive2/MyDrive/MasterThesis/taught-smart/-with-tra

In [ ]:
flushDrive()

Mounted at /content/drive2
